## 1. Tensorflow2 API로 모델 구성하기
- Sequential
- Functional은 Sequential의 보다 일반화된 개념
- Subclassing은 클래스로 구현된 기존의 모델을 상속받아 자신만의 모델을 만들어나가는 방식

### 1-1. TensorFlow2 Sequential Model

In [ ]:
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential()      # 모델 쌓기 용이 BUT 모델의 입력과 출력이 여러 개인 경우 부적합
model.add(__넣고싶은 레이어__)
model.add(__넣고싶은 레이어__)
model.add(__넣고싶은 레이어__)

model.fit(x, y, epochs=10, batch_size=32)

- 모델 쌓기 용이 
- 모델의 입력과 출력이 여러 개인 경우 부적합
- 반드시 입력 1, 출력 1

### 1-2. TensorFlow2 Functional API

In [ ]:
import tensorflow as tf
from tensorflow import keras

inputs = keras.Input(shape=(__원하는 입력값 모양__))
x = keras.layers.__넣고싶은 레이어__(관련 파라미터)(input)
x = keras.layers.__넣고싶은 레이어__(관련 파라미터)(x)
outputs = keras.layers.__넣고싶은 레이어__(관련 파라미터)(x)

model = keras.Model(inputs=inputs, outputs=outputs) # Seauential 과의 차이점, 더 일반적
model.fit(x,y, epochs=10, batch_size=32)

- Sequential Model을 활용하는 것과 다른 점은 바로 keras.Model을 사용한다는 점입니다. 그래서 Sequential Model을 쓰는 것보다 더 일반적인 접근인 것입니다. Sequential Model이란 사실 keras.Model을 상속받아 확장한 특수사례에 불과한 것이니까요. Functional API를 활용하면 앞서 배운 Sequential Model을 활용하는 것보다 더 자유로운 모델링을 진행할 수 있습니다.
- **함수형으로 모델을 구성한다는 것, 즉 입력과 출력을 규정함으로써 모델 전체를 규정한다**
- input, output 만으로 규정: 다중입력/출력 가능

### 1-3. TensorFlow2 Subclassing

In [ ]:
import tensorflow as tf
from tensorflow import keras

class CustomModel(keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.__정의하고자 하는 레이어__()
        self.__정의하고자 하는 레이어__()
        self.__정의하고자 하는 레이어__()

    def call(self, x):
        x = self.__정의하고자 하는 레이어__(x)
        x = self.__정의하고자 하는 레이어__(x)
        x = self.__정의하고자 하는 레이어__(x)

        return x

model = CustomModel()
model.fit(x,y, epochs=10, batch_size=32)

- Subclassing을 활용하면 제일 자유로운 모델링을 진행할 수 있습니다. 사실 본질적으로는 Functional한 접근과 차이가 없습니다. 이것은 keras.Model을 상속받은 모델 클래스를 만드는 것이기 때문입니다. 처음 만났던 Sequential Model도 따지고 보면 keras.Model을 상속받은 모델 클래스의 하나일 뿐입니다. keras.Model은 위와 같이 __init__()이라는 메소드 안에서 레이어 구성을 정의합니다. 그리고 call()이라는 메소드 안에서 레이어간 forward propagation을 구현합니다. 

## 2. MNIST
### 2-1. Sequential API 활용

In [7]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [8]:
# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis] # indexing으로 길이가 1인 새로운 축을 추가하기
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [9]:
# Sequential Model을 구성해주세요.
"""
Spec:
1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
3. Flatten 레이어
4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
"""

# 여기에 모델을 구성해주세요
model=keras.models.Sequential([keras.layers.Conv2D(32, 3, activation='relu'),
                              keras.layers.Conv2D(64, 3, activation='relu'),
                               keras.layers.Flatten(),
                               keras.layers.Dense(128, activation='relu'),
                               keras.layers.Dense(10, activation='softmax')
                              ])


In [10]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1016 - accuracy: 0.9681
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0348 - accuracy: 0.9893
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0198 - accuracy: 0.9935
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0126 - accuracy: 0.9959
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0084 - accuracy: 0.9971
313/313 - 2s - loss: 0.0540 - accuracy: 0.9872


[0.05404530093073845, 0.9872000217437744]

### 2-2. Functional API 활용

In [13]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [14]:
"""
Spec:
0. (28X28X1) 차원으로 정의된 Input
1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
3. Flatten 레이어
4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
"""

# 여기에 모델을 구성해 주세요.
inputs = keras.Input(shape=(28,28,1))

x = keras.layers.Conv2D(32,3, activation='relu')(inputs)
x = keras.layers.Conv2D(64,3, activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)

predictions = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions) # Seauential 과의 차이점, 더 일반적

In [15]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1083 - accuracy: 0.9671
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0344 - accuracy: 0.9890
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0189 - accuracy: 0.9934
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0120 - accuracy: 0.9959
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0089 - accuracy: 0.9971
313/313 - 0s - loss: 0.0630 - accuracy: 0.9844


[0.06298741698265076, 0.9843999743461609]

### 2-3. Subclassing 활용
keras.Model을 상속받은 클래스를 만드는 것입니다. __init__() 메소드 안에서 레이어를 선언하고, call() 메소드 안에서 forward propagation을 구현하는 방식임을 기억해 주세요. Functional 방식과 비교하자면, call()의 입력이 Input이고, call()의 리턴값이 Output이 되는 것입니다.

In [16]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [17]:
# Subclassing을 활용한 Model을 구성해주세요.
"""
Spec:
0. keras.Model 을 상속받았으며, __init__()와 call() 메소드를 가진 모델 클래스
1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
3. Flatten 레이어
4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
6. call의 입력값이 모델의 Input, call의 리턴값이 모델의 Output
"""

# 여기에 모델을 구성해주세요
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(32, 3, activation='relu')
        self.conv2 = keras.layers.Conv2D(64, 3, activation='relu')
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(128, activation='relu')
        self.fc2 = keras.layers.Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()


In [18]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1047 - accuracy: 0.9686
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0334 - accuracy: 0.9895
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0183 - accuracy: 0.9941
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0119 - accuracy: 0.9957
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0099 - accuracy: 0.9968
313/313 - 0s - loss: 0.0429 - accuracy: 0.9883


[0.04291442409157753, 0.9883000254631042]

## 3. CIFAR-100
### 3-1. Sequential API 활용

In [19]:
import tensorflow as tf
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

Exception: URL fetch failure on https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz: 500 -- Internal Server Error

In [20]:
# Sequential Model을 구성해주세요.
"""
Spec:
1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. pool_size가 2인 MaxPool 레이어
3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
4. pool_size가 2인 MaxPool 레이어
5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
"""

# 여기에 모델을 구성해주세요
model = keras.Sequential([
    keras.layers.Conv2D(16, 3, activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(100, activation='softmax')
])

In [21]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1816 - accuracy: 0.9461
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0530 - accuracy: 0.9835
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0360 - accuracy: 0.9884
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0257 - accuracy: 0.9915
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0186 - accuracy: 0.9944
313/313 - 2s - loss: 0.0301 - accuracy: 0.9902


[0.030098455026745796, 0.9901999831199646]

### 3-2. Functional API 활용

In [22]:
import tensorflow as tf
from tensorflow import keras

cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

169009152/169001437 [==============================] - 254s 2us/step


EOFError: Compressed file ended before the end-of-stream marker was reached

In [ ]:
# Functional API를 활용한 Model을 구성해주세요.
"""
Spec:
0. (32X32X3) 차원으로 정의된 Input
1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. pool_size가 2인 MaxPool 레이어
3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
4. pool_size가 2인 MaxPool 레이어
5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
"""

# 여기에 모델을 구성해주세요
inputs = keras.Input(shape=(32, 32, 3))

x = keras.layers.Conv2D(16, 3, activation='relu')(inputs)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Conv2D(32, 3, activation='relu')(x)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)
predictions = keras.layers.Dense(100, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

In [ ]:
# 모델 학습 설정
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

### 3-3. Subclassing 활용

In [ ]:
import tensorflow as tf
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

In [24]:
# Subclassing을 활용한 Model을 구성해주세요.
"""
Spec:
0. keras.Model 을 상속받았으며, __init__()와 call() 메소드를 가진 모델 클래스
1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. pool_size가 2인 MaxPool 레이어
3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
4. pool_size가 2인 MaxPool 레이어
5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
7. call의 입력값이 모델의 Input, call의 리턴값이 모델의 Output
"""

# 여기에 모델을 구성해주세요
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

In [25]:
# 모델 학습 설정

# 학습 관련 부분을 작성해주세요
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1701 - accuracy: 0.9496
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0515 - accuracy: 0.9844
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0348 - accuracy: 0.9892
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0260 - accuracy: 0.9911
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0194 - accuracy: 0.9938
313/313 - 0s - loss: 0.0325 - accuracy: 0.9893


[0.03248394653201103, 0.989300012588501]

## 4. GradientTape 활용

In [ ]:
# 동일하게 진행했던 부분
# 모델 학습 설정
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

- Forward Propagation 수행 및 중간 레이어값 저장
- Loss 값 계산
- 중간 레이어값 및 Loss를 활용한 체인룰(chain rule) 방식의 역전파(Backward Propagation) 수행
- 학습 파라미터 업데이트

=> model.fit()

Tensorflow에서 제공하는 tf.GradientTape는 위와 같이 순전파(forward pass) 로 진행된 모든 연산의 중간 레이어값을 tap에 기록하고, 이를 이용해 gradient를 계산한 후 tape를 폐기하는 기능을 수행합니다. 

In [26]:
import tensorflow as tf
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

# 모델 구성부분
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

UnpicklingError: pickle data was truncated

In [ ]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# tf.GradientTape()를 활용한 train_step
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [ ]:
import time
def train_model(batch_size=32):
    start = time.time()
    for epoch in range(5):
        x_batch = []
        y_batch = []
        for step, (x, y) in enumerate(zip(x_train, y_train)):
            if step % batch_size == batch_size-1:
                x_batch.append(x)
                y_batch.append(y)
                loss = train_step(np.array(x_batch), np.array(y_batch))
                x_batch = []
                y_batch = []
        print('Epoch %d: last batch loss = %.4f' % (epoch, float(loss)))
    print("It took {} seconds".format(time.time() - start))

train_model()

In [ ]:
# evaluation
prediction = model.predict(x_test, batch_size=x_test.shape[0], verbose=1)
temp = sum(np.squeeze(y_test) == np.argmax(prediction, axis=1))
temp/len(y_test)  # Accuracy